# K-Nearest Neighbors for Regression

Linear regression is not the only machine learning model that Ashenfelter could have fit.

In [1]:
import pandas as pd
import numpy as np

bordeaux_df = pd.read_csv("https://dlsun.github.io/pods/data/bordeaux.csv",
                          index_col="year")

In [2]:
bordeaux_df.head()

,price,summer,har,sep,win,age
year,,,,,,
1952,37.0,17.1,160,14.3,600,40
1953,63.0,16.7,80,17.3,690,39
1955,45.0,17.1,130,16.8,502,37
1957,22.0,16.1,110,16.2,420,35
1958,18.0,16.4,187,19.1,582,34


In [5]:
bordeaux_df.tail(15)

,price,summer,har,sep,win,age
year,,,,,,
1977,11.0,15.6,87,16.8,821,15
1978,27.0,15.8,51,17.4,763,14
1979,21.0,16.2,122,17.3,717,13
1980,14.0,16.0,74,18.4,578,12
1981,NaN,17.0,111,18.0,535,11
1982,NaN,17.4,162,18.5,712,10
1983,NaN,17.4,119,17.9,845,9
1984,NaN,16.5,119,16.0,591,8
1985,NaN,16.8,38,18.9,744,7


In [6]:
bordeaux_df.describe()

,price,summer,har,sep,win,age
count,27.000000,38.000000,38.000000,38.000000,38.000000,38.000000
mean,28.814815,16.715789,135.500000,17.392105,609.421053,19.631579
std,20.956066,0.781726,67.429944,1.439854,131.370866,11.336192
min,10.000000,15.000000,38.000000,14.300000,376.000000,1.000000
25%,14.000000,16.225000,86.250000,16.425000,535.250000,10.250000
50%,22.000000,16.700000,119.000000,17.300000,586.500000,19.500000
75%,35.000000,17.100000,171.000000,18.400000,713.500000,28.750000
max,100.000000,18.700000,292.000000,20.400000,845.000000,40.000000


In [7]:
bordeaux_df.shape

(38, 6)

In [8]:
# Split the data into training and test sets.
bordeaux_train = bordeaux_df.loc[:1980].copy()
bordeaux_test = bordeaux_df.loc[1981:].copy()

# Log transform the target.
bordeaux_train["log(price)"] = np.log(bordeaux_train["price"])
bordeaux_train.head()

,price,summer,har,sep,win,age,log(price)
year,,,,,,,
1952,37.0,17.1,160,14.3,600,40,3.610918
1953,63.0,16.7,80,17.3,690,39,4.143135
1955,45.0,17.1,130,16.8,502,37,3.806662
1957,22.0,16.1,110,16.2,420,35,3.091042
1958,18.0,16.4,187,19.1,582,34,2.890372


Let's focus on just two features for now: winter rainfall (**win**) and average summer temperature (**summer**). Let's plot the training data, using a color gradient to represent the target (**log(price)**). Notice how we can customize the color gradient using the `cmap=` argument. A list of the available colormaps can be found [here](https://matplotlib.org/examples/color/colormaps_reference.html).

In [9]:
import matplotlib.cm as cm

bordeaux_train.plot.scatter(x="win", y="summer", c="log(price)", 
                            cmap=cm.YlOrRd)

Notice how wines that are close on this scatterplot are of similar quality. This insight is the basis of the $k$-nearest neighbors algorithm for predicting wine quality. Suppose that we want to predict the quality of the 1986 vintage, represented by a blue star in the plot below. 

![](https://github.com/dlsun/pods/blob/master/05-Regression-Models/regression_neighbors.png?raw=1)

The $k=5$ points that are closest to this point in feature space are indicated by dotted lines. We can average the qualities of thse wines to obtain our prediction. All 5 of these points have a quality less than 3.0, so the 1986 vintage is also predicted to be of low quality.

The $k$-nearest neighbors regression algorithm can be summarized as follows:

1. Determine the $k$ closest points in the training data to the new point that you want to predict for, based on some distance metric on the features.
2. The predicted label of the new point is the mean (or median) of the labels of the $k$ closest points.

Let's implement this in code. First, we extract the training data and scale the features:

In [10]:
X_train = bordeaux_train[["win", "summer"]]
y_train = bordeaux_train["log(price)"]

# Standardize the features.
X_train_mean = X_train.mean()
X_train_sd = X_train.std()
X_train_st = (X_train - X_train_mean) / X_train_sd

In [11]:
X_train_st

,win,summer
year,,
1952,-0.065156,0.965533
1953,0.632329,0.352135
1955,-0.824640,0.965533
1957,-1.460127,-0.567960
1958,-0.204653,-0.107912
1959,-0.956387,1.578930
1960,1.198068,-0.107912
1961,1.717307,1.272231
1962,0.686578,-0.261262


Now, we get the features for the new observation (i.e., the 1986 vintage), standardizing it in the same way.

In [12]:
x_new = bordeaux_test.loc[1986, ["win", "summer"]]

x_new_st = (x_new - X_train_mean) / X_train_sd
x_new_st

win      -0.351900
summer   -0.261262
dtype: float64

In [ ]:
x_new_st

win      -0.351900
summer   -0.261262
dtype: float64

Now we calculate the (Euclidean) distances between the 1986 vintage and the vintages in the training data.

In [13]:
dists = np.sqrt(((X_train_st - x_new_st) ** 2).sum(axis=1))
dists

year
1952    1.259860
1953    1.159726
1955    1.314727
1957    1.149883
1958    0.212597
1959    1.936933
1960    1.557535
1961    2.575503
1962    1.038478
1963    0.983970
1964    1.976971
1965    1.412851
1966    2.007525
1967    1.180230
1968    0.395207
1969    0.320488
1970    0.765065
1971    0.772366
1972    2.004492
1973    1.898753
1974    0.085248
1975    0.922736
1976    2.288442
1977    2.269387
1978    1.729248
1979    1.203287
1980    0.474508
dtype: float64

Now, we sort the distances. The first 5 of these are the nearest neighbors. To get the year of these nearest neighbors, we get the index.

In [14]:
i_nearest = dists.sort_values().index[:5]
i_nearest

Int64Index([1974, 1958, 1969, 1968, 1980], dtype='int64', name='year')

We can look up these years in the training data.

In [15]:
bordeaux_train.loc[i_nearest]

,price,summer,har,sep,win,age,log(price)
year,,,,,,,
1974,11.0,16.3,184,16.2,574,18,2.397895
1958,18.0,16.4,187,19.1,582,34,2.890372
1969,12.0,16.5,244,16.6,575,23,2.484907
1968,11.0,16.2,292,16.4,610,24,2.397895
1980,14.0,16.0,74,18.4,578,12,2.639057


In [17]:
x_new

win       563.0
summer     16.3
Name: 1986, dtype: float64

To make a prediction for the price of the 1986 vintage, we average the sale prices of these 5-nearest neighbors.

In [21]:
y_train.loc[i_nearest[:3]].mean()

2.591057893494179

So the model predicts that the quality of the 1986 is about 2.56, which is well below the average quality.

Of course, the model above only had two features so it was easy to visualize the "nearest neighbors" on the scatterplot. The magic of $k$-nearest neighbors is that it still works when there are more features and the data is not possible to visualize.

## K-Nearest Neighbors in scikit-learn

Now let's see how to implement $k$-nearest neighbors in scikit-learn. Recall from the previous lesson that all scikit-learn models follow the three-step pattern:

1. Declare the model.
2. Fit the model to training data.
3. Use the model to predict on test data.

To fit a $k$-nearest neighbors model instead of a linear regression model, we only need to modify the first step. Instead of declaring a model of type `LinearRegression`, we define a model of type `KNeighborsRegressor`, specifying the value of $k$ as one of the parameters. Because `KNeighborsRegressor` works with distances, it is a good idea to scale the features before passing the features into the model. (Refer back to Chapter 3 for a full explanation.)

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

# Standardize the training and test data
scaler = StandardScaler()
X_train_st = scaler.fit_transform(X_train)
X_new_st = scaler.transform(pd.DataFrame([x_new])) # needs to be a DataFrame
y_train = bordeaux_train["log(price)"]

In [22]:
# Fit k-nearest neighbors
model = KNeighborsRegressor(n_neighbors=3)
model.fit(X=X_train_st, y=y_train)
model.predict(X=X_new_st)

array([2.59105789])

In [ ]:
2.591057893494179

In [23]:
X_train_st.shape

(27, 2)

In [24]:
nei = model.kneighbors_graph()

In [25]:
nei.toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


This is the same predicted value that we got by implementing $k$-nearest neighbors manually.

In the code above, we had to be careful to standardize the training data and the test data in exactly the same way before fitting the $k$-nearest neighbors model. Most machine learning models have many more preprocessing steps. As the preprocessing gets more complex, it is easy to accidentally omit one of the preprocessing steps. For this reason, scikit-learn provides a _Pipeline_ object, which simply chains together a sequence of preprocessing and model building steps. If we call `Pipeline.fit()` or `Pipeline.predict()` on the data, all of the steps will be applied to the data in a consistent manner.

In [26]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=5)
)

pipeline.fit(X=X_train, y=y_train)
pipeline.predict(X=pd.DataFrame([x_new]))

array([2.56202526])

## The K-Nearest Neighbors Regression Function

A predictive model is simply a function $f$ that maps feature values ${\bf x}$ to target values $y$. We can visualize $f$ when ${\bf x}$ consists of just a single feature, such as **age**. In the previous lesson, we saw that $f$ is just a line when the model is linear regression. What does $f$ look like when the model is a $k$-nearest neighbors regressor?

First, we extract the training data. There is no need to scale the features in this case because there is only one feature. (The point of scaling is to bring all of the variables to the same scale.

In [27]:
X_train = bordeaux_train[["age"]]
y_train = bordeaux_train["log(price)"]

Now we fit the $k$-nearest neighbor model as before. We do not need to standardize the feature in this case because there is only one feature. Standardizing is only useful when there are multiple features that we want to bring to the same scale.

In [38]:
# Fit k-nearest neighbors
model = KNeighborsRegressor(n_neighbors=4)
model.fit(X=X_train, y=y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                    weights='uniform')

In order to graph $f$, we need to evaluate the predictive model at a grid of feature values. Since age ranges from 12 to 40 in the training data, we create a grid of ${\bf x}$ values from 10 to 45, make predictions at these values, and plot these predictions as a curve.

In [39]:
# Define a grid of feature values.
X_new = pd.DataFrame()
X_new["age"] = np.linspace(10, 45, num=200)

# Make predictions at those feature values.
y_new_ = pd.Series(
    model.predict(X_new),
    index=X_new["age"]
)

# Plot the predictions.
bordeaux_train.plot.scatter(x="age", y="log(price)")
y_new_.plot.line()

Unlike the linear regression model, the $k$-nearest neighbor model is piecewise constant. For example, wines more than 37 years old all have the same 5-nearest neighbors, so the prediction is constant in that range.